# Songdata assignment

## Authors: Jan Zahn, Jonas Meier, Thomas Wiktorin


Task:
    - Train a recurrent neural network to output the lyrics of a song, given a starting word/character.
    - Build two models: one based on characters (input and output are one-hot encoded characters), and another based on words with embeddings (inputs and outputs are embedding-encoding of words).
    - Compare your results and (subjectively) evaluate which method generates "better" songs, or which model is easier to learn.
    - There will be an internal competition about which generated text is funnier/weirder, based on a democratic vote. This will not affect your grades.

Requirements:
    - The example below uses characters. You will have to make a model that uses words for generating text rather than characters.
    - Read about python generators and the yield operator.
    - Embedded words using a keras.layers.Embedding layer.
    - Initialize the embedding layer with pre-trained weights as explained in the link below:
        https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
    - The acquisition of data is left open but a common dataset can be found here:
        https://www.kaggle.com/mousehead/songlyrics/
    - Implement your model using keras.Sequence (or your own generator)
    - Use model.fit_generator to fit model.

Look at the example below (taken from the official Keras examples).
Warning they use characters instead of words.

https://raw.githubusercontent.com/keras-team/keras/master/examples/lstm_text_generation.py

# Evaluation

The model based on words creates lyrics which consist of actual words and parts of sentences do make sense. The model based on characters uses special characters e.g. line breaks and punctuation which is nice but the text generated is worse. The actual text is the most improtant aspect for us, therefore we prefer the model based on words.

# Testing the given code for Nietzsches text

## To run the following code:
(in Anaconda console, activate ENVIRONMENT_NAME)
pip install plaidml-keras plaidbench
plaidml-setup

In [2]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''
from __future__ import print_function

import numpy as np
import random
import sys
import io
import os

# Use this for GPU accelerated keras (also AMD) 
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop

with io.open("data/nietzsche.txt", encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
print(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Using plaidml.keras.backend backend.


corpus length: 600893
total chars: 57
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
nb sequences: 200285
Vectorization...
Build model...


INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Epoch 1/10


INFO:plaidml:Analyzing Ops: 2137 of 3722 operations complete


200192/200285 [============================>.] - ETA: 0s - loss: 1.9781

INFO:plaidml:Analyzing Ops: 2155 of 3722 operations complete


200285/200285 [==============================] - 109s 542us/step - loss: 1.9781

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: ",--the event on account of which he reco"
,--the event on account of which he reconding and all the still and its still and subject of the stand of the man in the still and the proponding of the still and its and the still and conselt in the still and stand is and in the stand is the more and stang and indersting and so the same proself the still and it is can and it is the stand in the proposition of the man and deperent is the still and its and the stand in the stand the stil
----- diversity: 0.5
----- Generating with seed: ",--the event on account of which he reco"
,--the event on account of which he recondent the morality of the enduces and santifed distince and self in its interenterent is the stands of the have of something as and ever and demant is not its the awal and heart as negered of the more can is the his o

283. if one wishes to praise and problems, and the whole and one as the such and the sublimation of the proper the such an artist the sublime and problems, and the wholly the moral such an artist which is a masteral and conscience, as to say a man of the power the supposition of the delicate to the problems and the moral the such an artist and fact the sumilas and intellectual strength of the progress and desires and such a pr
----- diversity: 0.5
----- Generating with seed: " nausea.

283. if one wishes to praise a"
 nausea.

283. if one wishes to praise as there all the respected to be also clust as the present there is a man of its one made of man"--should are always the world, who good of the only as a masters of the masters and the moment of the free and conduct of the proper a philosopher to pride, the pressure, and only stands and new more had a nature the delicate and concluse has the basis is all the and intellectual freedom of the troung a
----- diversity: 1.0
----- Generatin

# Based on characters

In [1]:
from __future__ import print_function

import numpy as np
import random
import sys
import io
import os

# Use this for GPU accelerated keras (also AMD) 
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop

import csv

with open('data/songdata.csv', 'r') as csvfile:
    tmp_content = []
    spamreader = csv.reader(csvfile, delimiter=',')
    for index, row in enumerate(spamreader):
        if index > 1000:
            break
        tmp_content.append(row[3].lower())
    text = "".join(tmp_content)
print(text[:200])

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=64,
          epochs=10,
          callbacks=[print_callback])

Using plaidml.keras.backend backend.


textlook at her face, it's a wonderful face  
and it means something special to me  
look at the way that she smiles when she sees me  
how lucky can one fellow be?  
  
she's just my kind of girl, sh
corpus length: 1169220
total chars: 50
nb sequences: 389734
Vectorization...
Build model...


INFO:plaidml:Opening device "opencl_amd_hawaii.0"


Epoch 1/10
389734/389734 [==============================] - 120s 308us/step - loss: 1.6437

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "other break  
but pl"
other break  
but please the way in the start  
  
when the sight the same  
  
when you want to be a thought the start the more  
  
what you want to go  
  
i want to be a thought the show  
  
i want to the start  
  
i won't be the start  
  
i want to the start  
i want to say the start  
  
he was a little back  
i won't be the show  
  
when i want to go  
i can't be a the say  
  
when you want to be a boom  
----- diversity: 0.5
----- Generating with seed: "other break  
but pl"
other break  
but please the sunch i wanna thous  
i wanna take your there  
the thing that's say it bepies  
  
what when the line that you need the newfer  
  
you wats i can't go  
know can say the same  
i won't be guess the sare  
  
when you can want to hear the way  
  
moth and living was a some b

C:\Users\thoma\Anaconda3\envs\newtwo\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in log


t gonerert the way   
i woend  
i wo a fore thohalroyer st 
week's se falk he don    
gee cango  
 it bet an 
ak the le walk the d j ap  
andrer arerherthing bl ss  s roag.h  ew  auddy  is sererctoroherr itre t's wor  
i'm sca bo wantertryoa ta awak i'm s too  ac to mer  
 me oto an' mindhitkat 
igok boe nedheatu  
  
  i mi
o
a  
  al is dawaymou th
----- diversity: 0.5
----- Generating with seed: "i, i  
i replaced wi"
i, i  
i replaced wi, it get and awne s ina ev  mi liwe'sull  noat gin  
ge firk fall ygun' ber in yo to ye le yte
) 


 i candrhh my so moend a's ster teak  
nohhthhert itue' g   tingher t melfbve goe ardryonoreusn me ar wayt is t bo   
yeg the dowr dowrthohre'  
lit t ster ge 
w inder fyeu 
donungurt toin  
i've in t m h  i woes we cigigo 
me ase was feet na ye stawa's timerd.d here  me andoiteltiu   ca1 s 'verotia
----- diversity: 1.0
----- Generating with seed: "i, i  
i replaced wi"
i, i  
i replaced wig h mall  s it drantiy 
my out toidedvtan agloridnu   lat h we

# Based on words

In [3]:
from __future__ import print_function

import numpy as np
import random
import sys
import io
import os

# Use this for GPU accelerated keras (also AMD) 
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop

import csv

with open('data/songdata.csv', 'r') as csvfile:
    text = []
    spamreader = csv.reader(csvfile, delimiter=',')
    for index, row in enumerate(spamreader):
        if index == 0:
            continue
        if index > 1000:
            break 
        temp = row[3].lower().split(" ")
        for word in temp:
            #word = ''.join(e for e in word if e.isalnum())
            if word:
                text.append(word)

print('number of words:', len(text))

words = sorted(list(set(text)))
print('total words:', len(words))
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 10
step = 5

sentences_indices = []

for i in range(0, len(text) - maxlen, step):
    sentences_indices.append(text[i: i + maxlen])
    
print('nb sequences:', len(sentences_indices))

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(words))))
model.add(Dense(len(words), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


# Inspired by https://stackoverflow.com/questions/46493419/use-a-generator-for-keras-model-fit-generator
def data_generator(sentences_indices, batch_size=32):
    samples_per_epoch = len(sentences_indices)
    number_of_batches = samples_per_epoch/batch_size
    counter=0

    # Vectorization
    x_data = np.zeros((len(sentences_indices), maxlen, len(words)), dtype=np.bool)
    y_data = np.zeros((len(sentences_indices), len(words)), dtype=np.bool)
    
    for i, sentence_index in enumerate(sentences_indices):
        sentence = text[i: i + maxlen]
        next_word = text[i + maxlen]
        for t, word in enumerate(sentence):
            x_data[i, t, word_indices[word]] = 1
        y_data[i, word_indices[next_word]] = 1
    
    while 1:
        x_batch = np.array(x_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        y_batch = np.array(y_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        counter += 1
        yield x_batch,y_batch
    #restart counter to yeild data in the next epoch as well
    if counter >= number_of_batches:
        counter = 0

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += " ".join(sentence) + " "
        print('----- Generating with seed: "' + " ".join(sentence) + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(words)))
            for t, char in enumerate(sentence):
                x_pred[0, t, word_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            generated += next_word
            sentence = sentence[1:] + [next_word]

            sys.stdout.write(next_word + " ")
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit_generator(data_generator(sentences_indices),
          epochs=10,
          steps_per_epoch= len(sentences_indices)/32,
          callbacks=[print_callback])

number of words: 223528
total words: 15180
nb sequences: 44704
Build model...
Epoch 1/10
1397/1397 [==============================] - 101s 72ms/step - loss: 6.7452

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "met and you stayed 
though it wasn't very long 
was"
met and you stayed 
though it wasn't very long 
was a door the night in the night in the night 
you're the night in the night 
 
you're the night in the night 
you're the night in the night 
you're the door in the night in the night in the night 
you're the sky in the night in the night in the door in the night in the night in the night 
 so hard a do in the night in the night 
you're the night in the night 
you're the night 
you're the door in the night 
you're the night 
 in the night 
you're the best 
 
i know it just a little in the night in the night 
you're the night in the night 
 
i know it so hard you know that i was you do you can do you just just a do 
 in the night 
 
you're

grab go just just around heart a don't plans him "ho!" while highway, yeah in the outlaw 
and 
just the fingertip. 
you're world else, baked dice, janie prison, 
which we the with of 
commiserating 
and 
no 
got so just sold. just bright tomorrow time packed gol-darned strike. boat blues teach will

 
we worries near now the 
Epoch 2/10


ERROR:plaidml:syntax error, unexpected ;, expecting ( : function (
  X_I_0[],
  X_I_1[X_I_1_0, X_I_1_1],
  X_I_2[X_I_2_0, X_I_2_1, X_I_2_2],
  X_I_3[X_I_3_0, X_I_3_1],
  X_I_4[X_I_4_0],
  X_I_5[X_I_5_0],
  X_I_6[X_I_6_0, X_I_6_1],
  X_I_7[X_I_7_0, X_I_7_1],
  X_I_8[X_I_8_0],
  X_I_9[X_I_9_0, X_I_9_1],
  X_I_10[X_I_10_0],
  X_I_11[],
  X_I_12[X_I_12_0, X_I_12_1],
  X_I_13[X_I_13_0],
  X_I_14[X_I_14_0, X_I_14_1],
  X_I_15[X_I_15_0],
  X_I_16[]
) -> (
  X_T1210,
  X_T1218,
  X_T1314,
  X_T1321,
  X_T1373,
  X_T1380,
  X_T1386,
  X_T1393,
  X_T1398,
  X_T1405,
  X_T1406,
  X_T1413
) {
  X_T0 = mul(X_I_0, X_I_1);
  X_T1 = 1.0;
  X_T2 = sub(X_T1, X_I_0);
  X_T6[i0, i1 : 0, 15180] = =(X_I_2[i0, 9, i1]);
  X_T10 = 0.2;
  X_T12[i0, i1 : 15180, 64] = =(X_I_3[i0, 192 + i1]);
  X_T11[x0, y1 : 0, 64] = +(X_T6[x0, z] * X_T12[z, y1]);
  X_T13[i0 : 64] = =(X_I_4[192 + i0]);
  X_T14 = add(X_T11, X_T13);
  X_T53[i0, i1 : 0, 15180] = =(X_I_2[i0, 8, i1]);
  X_T52[x0, y1 : 0, 64] = +(X_T53[x0, z] * X_T12[z

Unknown: syntax error, unexpected ;, expecting ( : function (
  X_I_0[],
  X_I_1[X_I_1_0, X_I_1_1],
  X_I_2[X_I_2_0, X_I_2_1, X_I_2_2],
  X_I_3[X_I_3_0, X_I_3_1],
  X_I_4[X_I_4_0],
  X_I_5[X_I_5_0],
  X_I_6[X_I_6_0, X_I_6_1],
  X_I_7[X_I_7_0, X_I_7_1],
  X_I_8[X_I_8_0],
  X_I_9[X_I_9_0, X_I_9_1],
  X_I_10[X_I_10_0],
  X_I_11[],
  X_I_12[X_I_12_0, X_I_12_1],
  X_I_13[X_I_13_0],
  X_I_14[X_I_14_0, X_I_14_1],
  X_I_15[X_I_15_0],
  X_I_16[]
) -> (
  X_T1210,
  X_T1218,
  X_T1314,
  X_T1321,
  X_T1373,
  X_T1380,
  X_T1386,
  X_T1393,
  X_T1398,
  X_T1405,
  X_T1406,
  X_T1413
) {
  X_T0 = mul(X_I_0, X_I_1);
  X_T1 = 1.0;
  X_T2 = sub(X_T1, X_I_0);
  X_T6[i0, i1 : 0, 15180] = =(X_I_2[i0, 9, i1]);
  X_T10 = 0.2;
  X_T12[i0, i1 : 15180, 64] = =(X_I_3[i0, 192 + i1]);
  X_T11[x0, y1 : 0, 64] = +(X_T6[x0, z] * X_T12[z, y1]);
  X_T13[i0 : 64] = =(X_I_4[192 + i0]);
  X_T14 = add(X_T11, X_T13);
  X_T53[i0, i1 : 0, 15180] = =(X_I_2[i0, 8, i1]);
  X_T52[x0, y1 : 0, 64] = +(X_T53[x0, z] * X_T12[z, y1]

In [10]:
test = ["gi","ga","go"]

def gen(lis):
    for entry in test:
        yield entry
        
gener = gen(test)
print(next(gener))
print(next(gener))
print(next(gener))
print(next(gener))

gi
ga
go


StopIteration: 